## Setup and important functions


In [1]:
import os
import sys
dir_project = os.path.dirname(os.getcwd())
path = os.path.join(dir_project, 'src')
if path not in sys.path:
    sys.path.append(path)

current_directory = os.getcwd()
#os.chdir(os.path.join(dir_project, 'src'))
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\nilsh\my_projects\SeqLP\jupyter_scripts


In [2]:
import pandas as pd
from seqlp.visualize.supervised_ml import DataPipeline, SupervisedML
import glob
from seqlp.use_model import AnalyseModel
from Bio import SeqIO
import time
import numpy as np
import matplotlib.pyplot as plt


c:\Users\nilsh\my_projects\SeqLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
nanobody_delphia_no_ag_filter = pd.read_excel(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\nanobody_delphia_no_ag_filter.xlsx")
nanobody_delphia_ag_filter = pd.read_excel(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\nanobody_delphia_tidied.xlsx")
binding_data = pd.read_excel(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\all_binding_data.xlsx")
sequencing_report = pd.read_csv(r"C:\Users\nilsh\my_projects\ExpoSeq\my_experiments\max_new\sequencing_report.csv",)

Models

In [4]:
nanobody_model = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
esm_small = r"facebook/esm2_t6_8M_UR50D"


In [ ]:


def read_fasta(fasta_file):
    sequences = []
    headers = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))
        headers.append(record.id)
    return sequences, headers

from Bio.Seq import Seq
def translate_nucleotide_to_amino_acid(nucleotide_sequence):
    seq = Seq(nucleotide_sequence)
    return str(seq.translate())    


In [ ]:
Data = DataPipeline(model = r"C:\Users\nilsh\my_projects\ExpoSeq\models\nanobody_full",
             path_seq_report = r"C:\Users\nilsh\my_projects\ExpoSeq\my_experiments\max_new\sequencing_report.csv",
             no_sequences = 100000,  # take basically all sequences
             pca_components=40)


In [ ]:
sequencing_report = Data.init_sequencing_report
sequencing_report["v_gene"] = sequencing_report['allVHitsWithScore'].str.split('*').str[0]
experiments = sequencing_report["Experiment"].unique().tolist()
v_family = sequencing_report["v_gene"].tolist()
sequencing_report["full_seq"] = Data.full_sequences
print(f"No. of sequences in report: {sequencing_report.shape[0]}")

In [ ]:
training_data = pd.read_csv(r'c:\Users\nilsh\OneDrive\Desktop\master_thesis\train_model\concatenatednanobody_full_train.csv.gz', compression='gzip')
sequences = training_data.iloc[:, 0]
sequences = sequences.str.replace(' ', '')
print(f"No. of training sequences: {training_data.shape[0]}")


In [ ]:
mask = sequencing_report['full_seq'].isin(sequences)

# Step 4: Filter the DataFrame
sequencing_report = sequencing_report[~mask]
print(f"No. of sequences in report after filtering training sequences: {sequencing_report.shape[0]}")

### Perplexity Analysis

The goal is to find a metric which captures how decisive or confused the model is. We will use perplexity for this which is 2 ** Entropy.
The highest entropy for this task is 20. That means that each amino acid per position can appear with equal probability which is very bad.

Cons of this metric:
- not good for final evaluation, since it just measures the model's confidence not its accuracy
- A model with a lower perplexity can still be worse because it can just be very decisive but then in the clsutering would kinda fail to represent the sequences meaningfully.

-> your model does not output logits, so it does not really make sense to do that

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_perplexity(perplexities):
    mean_perplexity = np.mean(perplexities)
    median_perplexity = np.median(perplexities)
    std_dev = np.std(perplexities)

    # Visualization
    plt.figure(figsize=(10, 5))
    plt.hist(perplexities, bins=30, alpha=0.7, color='blue', label='Perplexity')
    plt.axvline(mean_perplexity, color='r', linestyle='dashed', linewidth=1, label=f'Mean: {mean_perplexity:.2f}')
    plt.axvline(median_perplexity, color='g', linestyle='dashed', linewidth=1, label=f'Median: {median_perplexity:.2f}')
    plt.title('Distribution of Perplexity Scores')
    plt.xlabel('Perplexity')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

In [ ]:
from seqlp.visualize.load_model import LoadModel

NanobodyModel = LoadModel(r"C:\Users\nilsh\my_projects\ExpoSeq\models\nanobody_full")
sequences = sequencing_report["full_seq"].tolist()
perplexities = []
for sequence in sequences:
    perplexity = NanobodyModel._get_perplexity(sequence)
    perplexities.append(perplexity)
perplexities = np.array(perplexities)



### COmpare model architectures based on no of components to reach 90%

In [ ]:
sequences = pd.read_csv(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\training_data\val.csv", nrows = 100000)["sequence"].tolist()
Analyse = AnalyseModel(r"c:\Users\nilsh\OneDrive\Desktop\results_thesis\models\t6_320_lastlayer_2106_seqs")
n_comp_collected = []
sequence_lengths = [100, 500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for length in sequence_lengths:
    chunk = sequences[:length]
    n_comp_collected.append(Analyse.no_components_for_sequences(chunk))
    

## Ideas for clustering

- Show how model clusters nanobodies with binding data. This model should do that gradually while the others should have problems with that


Analyse different layers

In [ ]:

full_sequences, experiments = DataPipeline.wrangle_report(sequencing_report)
import numpy as np
full_sequences = np.array(full_sequences).reshape(-1, 12050).T
experiments = np.array(experiments).reshape(-1, 12050).T

full_sequences.shape

# Create a DataFrame from the arrays
df = pd.DataFrame({
    "CDR3": sequencing_report["aaSeqCDR3"],
    'Full Sequences': full_sequences.flatten(),  # Flattening in case the array is 2D but should be 1D per column
    'Experiments': experiments.flatten(),

})
df_filtered = df.drop_duplicates(subset = ["CDR3"], keep = "last")
# This creates a mask that is True for rows where 'Full Sequences' does not contain 'region_not_covered'
mask = ~df['Full Sequences'].str.contains('region_not_covered')

# Apply the mask to the DataFrame to keep only the rows where the condition is True
df_filtered = df[mask]

df_filtered = df_filtered.groupby("Experiments").head(200)
df_filtered["Experiments"].unique()
experiments = df_filtered["Experiments"].tolist()
full_sequences = df_filtered["Full Sequences"].tolist()
cdr3 = df_filtered["CDR3"].tolist()


In [ ]:
cmap = plt.get_cmap('Set2')

# Number of colors in Set2
n_colors = cmap.N

# Retrieve each color from the colormap
colors = [cmap(i / float(n_colors - 1)) for i in range(n_colors)]
color_list = [colors[0], colors[7], colors[1], colors[2], colors[3], colors[4], colors[5]]

In [ ]:
color_list = [colors[0], colors[7], colors[1], colors[2], colors[3], colors[6], colors[5]]
def sequence_label(model_path, picture_path, title_extension, sequences, labels,):
    from transformers import RoFormerTokenizer, RoFormerModel
    if model_path == "alchemab/antiberta2-cssp":
        tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
        model = RoFormerModel.from_pretrained("alchemab/antiberta2-cssp")
        Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )
    else:
        Analyse = AnalyseModel(model_path)
    reduced_X = Analyse.embed_cluster_label(sequences, labels, explained_variance_threshold = 0.9,
                                            size_points = 15, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, 
                                            cmap = "Set2", title = title_extension, color_list = color_list)
    reduced_X.to_csv(f"{picture_path}.csv")

    Analyse.save_in_plots(f"{picture_path}.png")

In [ ]:
sequences = nanobody_delphia_ag_filter["Full sequence"].tolist()
labels = nanobody_delphia_ag_filter["Ag1_modified"].tolist()

In [ ]:
sequence_label(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\models\t12_320_lastlayer_2106_seqs", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\measure_pca_little", 
               "6 attention layers and 320 nodes in last hidden layer", sequences, labels)

Prepare binding data:

Embed specific targets

In [ ]:


def sequence_label(model_path, picture_path, title_extension):
    signals = nanobody_delphia_ag_filter["Ag1_Raw signals"].tolist()
    labels = nanobody_delphia_ag_filter["Ag1_modified"].tolist()
    sequences = nanobody_delphia_ag_filter["Full sequence"].tolist()
    from transformers import RoFormerTokenizer, RoFormerModel
    if model_path == "alchemab/antiberta2-cssp":
        tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
        model = RoFormerModel.from_pretrained("alchemab/antiberta2-cssp")
        Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )
    else:
        Analyse = AnalyseModel(model_path)
    reduced_X = Analyse.embed_cluster_label(sequences, labels, explained_variance_threshold = 0.9,size_points = 50, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, cmap = "Set2", title = f"Embedding of nanobody sequences with {title_extension}")
    reduced_X["Name VHH"] = nanobody_delphia_ag_filter["Name VHH"]
    reduced_X.to_csv(f"{picture_path}.csv")

    Analyse.save_in_plots(f"{picture_path}.png")

In [ ]:
sequence_label(r"facebook/esm2_t6_8M_UR50D", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\embedding_esm_t6_90%", "Esm-2b with 6 layers as baseline model", )


In [ ]:
sequence_label(r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\embedding_self_trained_90%", "trained model")


In [ ]:
sequence_label(r"alchemab/antiberta2-cssp", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\embedding_antiberta_90%", r"Antiberta2")
sequence_label(r"facebook/esm2_t6_8M_UR50D", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\embedding_esm_t6_90%", "Esm-2b with 6 layers as baseline model")


In [ ]:
sequence_label(r"facebook/esm2_t36_3B_UR50D", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\embedding_esm_t36_90%", "Esm-2b with 36 layers as baseline model")

Just embed whole venoms

In [ ]:
nanobody_delphia = nanobody_delphia_no_ag_filter.dropna(subset=['Ag1_modified'])
print(nanobody_delphia["Ag1_modified"].unique())
print(nanobody_delphia.shape)
nanobody_delphia = nanobody_delphia[nanobody_delphia["Ag1_modified"] != "SVSP"]
nanobody_delphia = nanobody_delphia[nanobody_delphia["Ag1_modified"] != "Dv4"]

In [ ]:
cmap = plt.get_cmap('Set2')
cmap_2 = plt.get_cmap('Set1')
# Number of colors in Set2
n_colors = cmap.N
n_colors_2 = cmap_2.N
# Retrieve each color from the colormap
colors_2 = [cmap_2(i / float(n_colors_2 - 1)) for i in range(n_colors_2)]

colors = [cmap(i / float(n_colors - 1)) for i in range(n_colors)]
color_list = [colors_2[0], colors[5], colors_2[1], colors[2], colors[1],  colors[4], colors_2[3]]

In [ ]:
model_path = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
picture_path = "all_venoms_self_trained_90%"
title_extension = "Embedding of "
signals = nanobody_delphia["Ag1_Raw signals"].tolist()
labels = nanobody_delphia["Ag1_modified"].tolist()
sequences = nanobody_delphia["Full sequence"].tolist()
from transformers import RoFormerTokenizer, RoFormerModel
if model_path == "alchemab/antiberta2-cssp":
    tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
    model = RoFormerModel.from_pretrained("alchemab/antiberta2-cssp")
    Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )
else:
    Analyse = AnalyseModel(model_path)
reduced_X = Analyse.embed_cluster_label(sequences, labels, explained_variance_threshold = 0.9,size_points = 35, n_neighbors = 15, min_dist = 0.1, alpha = 0.9, cmap = "Set2", title = f"Embedding of nanobody sequences selected against different venoms or toxins", color_list = color_list)
reduced_X.to_csv(f"{picture_path}.csv")

Analyse.save_in_plots(f"{picture_path}.png")

In [ ]:
cmap = plt.get_cmap('Set2')

# Number of colors in Set2
n_colors = cmap.N

# Retrieve each color from the colormap
colors = [cmap(i / float(n_colors - 1)) for i in range(n_colors)]
color_list = [colors[0], colors[7], colors[1], colors[2], colors[3], colors[4], colors[5]]


In [ ]:
sequencing_report = pd.read_csv(r"C:\Users\nilsh\my_projects\ExpoSeq\my_experiments\max_new\sequencing_report.csv",)


In [ ]:
# Create a DataFrame from the arrays
df = pd.DataFrame({
    "CDR3": sequencing_report["aaSeqCDR3"],
    'Full Sequences': full_sequences.flatten(),  # Flattening in case the array is 2D but should be 1D per column
    'Experiments': experiments.flatten(),

})
df_filtered = df.drop_duplicates(subset = ["CDR3"], keep = "last")
# This creates a mask that is True for rows where 'Full Sequences' does not contain 'region_not_covered'
mask = ~df['Full Sequences'].str.contains('region_not_covered')

# Apply the mask to the DataFrame to keep only the rows where the condition is True
df_filtered = df[mask]


In [ ]:
df_filtered = df_filtered.groupby("Experiments").head(200)
df_filtered["Experiments"].unique()
experiments = df_filtered["Experiments"].tolist()
full_sequences = df_filtered["Full Sequences"].tolist()
cdr3 = df_filtered["CDR3"].tolist()



In [ ]:

color_list = [colors[0], colors[7], colors[1], colors[2], colors[3], colors[6], colors[5]]
def sequence_label(model_path, picture_path, title_extension, sequences, labels,):
    from transformers import RoFormerTokenizer, RoFormerModel
    if model_path == "alchemab/antiberta2-cssp":
        tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
        model = RoFormerModel.from_pretrained("alchemab/antiberta2-cssp")
        Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )
    else:
        Analyse = AnalyseModel(model_path)
    reduced_X = Analyse.embed_cluster_label(sequences, labels, explained_variance_threshold = 0.9,
                                            size_points = 15, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, 
                                            cmap = "Set3", title = f"Embedding of CDR3 sequences with {title_extension}", color_list = color_list)
    reduced_X.to_csv(f"{picture_path}.csv")

    Analyse.save_in_plots(f"{picture_path}.png")

In [ ]:


sequence_label(r"facebook/esm2_t6_8M_UR50D", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\embedding_baseline_max_data_cdr3", 
               "Esm-2b with 6 layers as baseline model", cdr3, experiments)


In [ ]:
sequence_label(r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model", r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\validation_embedding\embedding_nanobody_model_max_data_cdr3", 
               "trained model", cdr3, experiments)

In [ ]:
from Bio.Seq import Seq
def translate_nucleotide_to_amino_acid(nucleotide_sequence):
    seq = Seq(nucleotide_sequence)
    return str(seq.translate())    


model_path = r"alchemab/antiberta2-cssp"
name_data = "embedding_antiberta_non_binding"
nanobody_delphia = pd.read_excel(r"C:\Users\nilsh\my_projects\SeqLP\jupyter_scripts\antibody_tidied.xlsx")
consensus_binding = nanobody_delphia[nanobody_delphia["Ag1"] == "scNTX"]
sequences = consensus_binding["Full sequence"].tolist()
signals = consensus_binding["Ag1_Raw signals"].tolist()
faqs_data = pd.read_csv(r"C:\Users\nilsh\my_projects\ExpoSeq\my_experiments\max_new\sequencing_report.csv")
faqs_data = faqs_data.groupby("Experiment").head(200)
binder_unlabeled = faqs_data[faqs_data["Experiment"] == "cLNTX_bind"]
non_binder = faqs_data[faqs_data["Experiment"] == "cLNTX_non-bind"]
seq_binder = non_binder["targetSequences"].apply(translate_nucleotide_to_amino_acid).tolist()
seq_non_binder = non_binder["targetSequences"].apply(translate_nucleotide_to_amino_acid).tolist()
binding_values_non_binder = len(seq_non_binder) * [0]
binding_values_binder = len(seq_binder) * [0]
all_binding =  binding_values_binder +binding_values_non_binder + signals
all_sequences =  seq_binder + seq_non_binder + sequences
labels =  len(seq_binder) * ["cLNTX binder"] + len(binding_values_non_binder) * ["Non Binder"] + len(signals) * ["cLNTX binder"]

from transformers import RoFormerTokenizer, RoFormerModel
if model_path == "alchemab/antiberta2-cssp":
    tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
    model = RoFormerModel.from_pretrained("alchemab/antiberta2-cssp")
    Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )
else:
    Analyse = AnalyseModel(model_path)
reduced_X = Analyse.embed_cluster_sequences(all_sequences, labels, all_binding, explained_variance_threshold = 0.9, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, cmap = "inferno", title = "Embedding of nanobody sequences with Antiberta 2.")
#reduced_X.to_csv(f"{name_data}.csv")

Analyse.save_in_plots(f"{name_data}.png")

Validate with binding data


In [ ]:
full_sequences, experiments = DataPipeline.wrangle_report(sequencing_report)
import numpy as np
full_sequences = np.array(full_sequences).reshape(-1, 12050).T
experiments = np.array(experiments).reshape(-1, 12050).T

full_sequences.shape
df = pd.DataFrame({
    "CDR3": sequencing_report["aaSeqCDR3"],
    'Full sequence': full_sequences.flatten(),  # Flattening in case the array is 2D but should be 1D per column
    'Experiments': experiments.flatten(),

})
df_filtered = df.drop_duplicates(subset = ["CDR3"], keep = "last")
# This creates a mask that is True for rows where 'Full Sequences' does not contain 'region_not_covered'
mask = ~df['Full sequence'].str.contains('region_not_covered')

# Apply the mask to the DataFrame to keep only the rows where the condition is True
df_filtered = df[mask]
df_filtered = df_filtered.groupby("Experiments").head(200)
bind_ngs = df_filtered.merge(binding_data, on = "Full sequence", how = "outer")

In [ ]:
bind_ngs.shape

In [ ]:

full_sequences = bind_ngs["Full sequence"].tolist()
bind_ngs = bind_ngs.fillna(0)
signals = bind_ngs[["⍺-BuTx", "aCBTX"]]
aCBTX = bind_ngs["aCBTX"].tolist()



In [ ]:
bind_ngs["Experiments"].unique()


In [ ]:
bind_ngs.loc[bind_ngs['aCBTX'] > 10000, 'Experiments'] = 'aCBTX_binder'
bind_ngs.loc[bind_ngs['⍺-BuTx'] > 10000, 'Experiments'] = 'aBGTx_binder'
bind_ngs = bind_ngs.loc[bind_ngs['Experiments'] != 0]

In [ ]:
bind_ngs["Experiments"].unique()

In [ ]:
labels = bind_ngs["Experiments"].tolist()
full_sequences = bind_ngs["Full sequence"].str.slice(0, 115).tolist()

In [ ]:
model_path = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
name_data = "aCBTX_max_data"
from transformers import RoFormerTokenizer, RoFormerModel
if model_path == "alchemab/antiberta2-cssp":
    tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
    model = RoFormerModel.from_pretrained("alchemab/antiberta2-cssp")
    Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )
else:
    Analyse = AnalyseModel(model_path)
reduced_X = Analyse.embed_cluster_label(full_sequences, labels, explained_variance_threshold = 0.9,
                                        size_points = 15, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, 
                                        cmap = "tab10", title = f"Embedding of NGS sequences with identified binders against aBTX and aCBTX")
#reduced_X.to_csv(f"{name_data}.csv")

Analyse.save_in_plots(f"{name_data}.png")

## Time Measurements


In [ ]:

def time_process(model_path, no_sequences = 1000, region_name = None):
    start_time = time.time()
    faqs_data = pd.read_csv(r"C:\Users\nilsh\my_projects\ExpoSeq\my_experiments\max_new\sequencing_report.csv").head(no_sequences)
    if region_name != None:
        sequences = faqs_data[region_name].tolist()
    else:
        sequences = faqs_data["targetSequences"].apply(translate_nucleotide_to_amino_acid).tolist()
    from transformers import RoFormerTokenizer, RoFormerModel
    if model_path == "alchemab/antiberta2-cssp":
        tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
        model = RoFormerModel.from_pretrained("alchemab/antiberta2-cssp")
        Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )
    else:
        Analyse = AnalyseModel(model_path)
    X = Analyse.ModelSets._get_embeddings_parallel(sequences, )
    end_time = time.time()

    elapsed_time = end_time - start_time
    return elapsed_time

In [ ]:
time_process(r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model", 1000, region_name = None )

In [ ]:
elapsed_time = time_process(r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model", 1000, region_name = None )

In [ ]:
models = ["alchemab/antiberta2-cssp", r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model", r"facebook/esm2_t6_8M_UR50D", r"facebook/esm2_t30_150M_UR50D"]
no_seq = 1000
all_times = []
region = None
folds = 5
for model in models:
    fold_model = []
    for fold in range(folds):
        elapsed_time = time_process(model, no_seq, region_name = None )
        fold_model.append(elapsed_time)
    all_times.append(fold_model)

means = [np.mean(sample_data) for sample_data in all_times]
std_devs = [np.std(sample_data) for sample_data in all_times]

names = ["Antiberta2", "Self-trained model", "Esm-2b with 6 layers", "Esm-2b with 30 layers"]
assert len(names) == len(models), "The number of models and names must be the same"
Analyse = AnalyseModel(r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model")
Analyse.make_figure()
Analyse.ax.bar(names, means, yerr = std_devs, capsize = 5, color = "skyblue", alpha = 0.7)
Analyse.update_plot()
Analyse.ax.set_ylabel("Time in seconds", **Analyse.font_settings)
Analyse.ax.set_xlabel("Model",  **Analyse.font_settings)
Analyse.ax.set_title("Computing time for embedding of 1000 sequences", pad = 20, **Analyse.font_settings)
Analyse.save_in_plots("time_comparison.png")


In [ ]:
import numpy as np
means = [np.mean(sample_data) for sample_data in all_times]
std_devs = [np.std(sample_data) for sample_data in all_times]
print(np.mean([
    125.92300152778625, 125.60665917396545, 124.9071409702301, 125.5185558795929]))
print(np.std([125.92300152778625, 125.60665917396545, 124.9071409702301, 125.5185558795929]))

### Calculating Perplexity

In [ ]:

model_path = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
if model_path == "alchemab/antiberta2-cssp":
    from transformers import RoFormerTokenizer, RoFormerForMaskedLM
    tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
    model = RoFormerForMaskedLM.from_pretrained("alchemab/antiberta2-cssp")
    Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )     
else:
    Analyse = AnalyseModel(model_path, load_masked_lm=True)
sequences = pd.read_csv(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\training_data\val.csv", nrows = 10000)["sequence"].tolist()
all_seqs = []
for seq in sequences:
    if len(seq) < 240: # there are some sequence lengths in the data that dont make sense in the context of nanobodies. 240 because there are spaces in between each tokens
        pass
    else:
        all_seqs.append(seq)
    if len(all_seqs) == 1000:
        break
ppl = Analyse.calculate_perplexity(all_seqs)
print(ppl)


In [ ]:
model_path =  r"facebook/esm2_t6_8M_UR50D"
if model_path == "alchemab/antiberta2-cssp":
    from transformers import RoFormerTokenizer, RoFormerForMaskedLM
    tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2-cssp")
    model = RoFormerForMaskedLM.from_pretrained("alchemab/antiberta2-cssp")
    Analyse = AnalyseModel("alchemab/antiberta2-cssp", model, tokenizer )     
else:
    Analyse = AnalyseModel(model_path, load_masked_lm=True)
sequences = pd.read_csv(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\training_data\val.csv", nrows = 1000)["sequence"].tolist()
all_seqs = []
for seq in sequences:
    if len(seq) < 240:
        pass
    else:
        all_seqs.append(seq)
    if len(all_seqs) == 1000:
        break
ppl = Analyse.calculate_perplexity(all_seqs)
print(ppl)

 The model with lower perplexity might be heavily optimized towards predicting the next amino acid accurately, focusing narrowly on features that are directly predictive of the next outcome. This can lead to a situation where the embeddings, while effective for prediction, may not capture broader or more nuanced relationships between different types of amino acids.

 A reason for this can be that the model cannot distinguish between the cdr and other regions

In [ ]:
model_path =  r"alchemab/antiberta2"
if model_path == "alchemab/antiberta2":
    from transformers import RoFormerTokenizer, RoFormerForMaskedLM
    tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2")
    model = RoFormerForMaskedLM.from_pretrained("alchemab/antiberta2")
    Analyse = AnalyseModel("alchemab/antiberta2", model, tokenizer )     
else:
    Analyse = AnalyseModel(model_path, load_masked_lm=True)
sequences = pd.read_csv(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\training_data\val.csv", nrows = 1000)["sequence"].tolist()
all_seqs = []
for seq in sequences:
    if len(seq) < 240:
        pass
    else:
        all_seqs.append(seq)
    if len(all_seqs) == 1000:
        break
ppl = Analyse.calculate_perplexity(all_seqs, pad_token_id=0) # token id is 0, https://huggingface.co/alchemab/antiberta2/blob/main/vocab.txt
print(ppl)

In [ ]:
model_path =  r"facebook/esm2_t30_150M_UR50D"
if model_path == "alchemab/antiberta2":
    from transformers import RoFormerTokenizer, RoFormerForMaskedLM
    tokenizer = RoFormerTokenizer.from_pretrained("alchemab/antiberta2")
    model = RoFormerForMaskedLM.from_pretrained("alchemab/antiberta2")
    Analyse = AnalyseModel("alchemab/antiberta2", model, tokenizer )     
else:
    Analyse = AnalyseModel(model_path, load_masked_lm=True)
sequences = pd.read_csv(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\training_data\val.csv", nrows = 1000)["sequence"].tolist()
all_seqs = []
for seq in sequences:
    if len(seq) < 240:
        pass
    else:
        all_seqs.append(seq)
    if len(all_seqs) == 1000:
        break
ppl = Analyse.calculate_perplexity(all_seqs) # token id is 0, https://huggingface.co/alchemab/antiberta2/blob/main/vocab.txt
print(ppl)

## Check CDR3 positional importance

In [ ]:
nanobody_delphia.head(10)

In [ ]:
import pandas as pd 

nanobody_delphia = pd.read_excel(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\nanobody_delphia_tidied.xlsx")
nanobody_delphia = nanobody_delphia.dropna(subset=['CDR3', 'Full sequence'])


def pad_sequence(subseq, fullseq):
    start_index = fullseq.find(subseq)
    if start_index == -1:
        # If subsequence not found, return padded version of the full sequence
        return ""
    
    # Create a list of <PAD> for each character in full sequence
    padded_sequence = ['<pad>' for _ in fullseq]
    
    # Replace the <PAD> tokens with the subsequence at the correct position
    for i in range(len(subseq)):
        padded_sequence[start_index + i] = subseq[i]
    
    # Join the sequence with space
    return ' '.join(padded_sequence)

# Apply the function to each row in DataFrame
sequencing_report = pd.read_csv(r"C:\Users\nilsh\my_projects\ExpoSeq\my_experiments\max_new\sequencing_report.csv",)
mask = ~sequencing_report['targetSequences'].str.contains('region_not_covered')

# Apply the mask to the DataFrame to keep only the rows where the condition is True
sequencing_report = sequencing_report[mask]
sequencing_report["full_seq"] = sequencing_report["targetSequences"].apply(translate_nucleotide_to_amino_acid)
sequencing_report["padded_sequence"] = sequencing_report.apply(lambda x: pad_sequence(x['aaSeqCDR3'], x['full_seq']), axis=1)
mask = sequencing_report['padded_sequence'].str.len() > 0
report = sequencing_report[mask]

#nanobody_delphia['PaddedSequence'] = nanobody_delphia.apply(lambda x: pad_sequence(x['CDR3'], x['Full sequence']), axis=1)

In [ ]:
sequencing_report["padded_sequence"][0]

In [ ]:
report = report.groupby("Experiment").head(200)
padded_sequences = report["padded_sequence"].tolist()
labels = report["Experiment"].tolist()


In [ ]:
unpadded_sequences = report["aaSeqCDR3"].tolist()

In [ ]:
picture_path = "unpadded_cdr3_faqs_data_25n"
title_extension = "specifically trained model"
model_path = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
Analyse = AnalyseModel(model_path)
reduced_X = Analyse.embed_cluster_label(unpadded_sequences, labels, explained_variance_threshold = 0.9, n_neighbors = 25, min_dist = 0.1, alpha = 0.8, cmap = "Set2", title = f"Unpadded CDR3 sequences with n = 25", color_list = color_list)
reduced_X.to_csv(f"{picture_path}.csv")
Analyse.save_in_plots(f"{picture_path}.png")

In [ ]:
print(nanobody_delphia.shape[0])
nanobody_delphia["Ag1_modified"].unique()

In [ ]:
padded_sequences = nanobody_delphia["PaddedSequence"].tolist()
labels = nanobody_delphia["Ag1_modified"].tolist()


In [ ]:
picture_path = "position_cdr_importance"
title_extension = "specifically trained model"
model_path = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
Analyse = AnalyseModel(model_path)
reduced_X = Analyse.embed_cluster_label(padded_sequences, labels, explained_variance_threshold = 0.9, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, cmap = "Set2", title = f"Embedding of position specific padded CDR3 sequences.")
reduced_X.to_csv(f"{picture_path}.csv")

Analyse.save_in_plots(f"{picture_path}.png")

In [ ]:
Analyse = AnalyseModel(r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model", load_masked_lm=True)
ppl = Analyse.calculate_perplexity(padded_sequences)
print(ppl)

In [ ]:
picture_path = "position_cdr_dumped"
title_extension = "specifically trained model"
unpadded_sequences = nanobody_delphia["CDR3"].tolist()
model_path = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
Analyse = AnalyseModel(model_path)
reduced_X = Analyse.embed_cluster_label(unpadded_sequences, labels, explained_variance_threshold = 0.9, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, cmap = "Set2", title = f"Embedding of unpadded CDR3 sequences with {title_extension}")
reduced_X.to_csv(f"{picture_path}.csv")

Analyse.save_in_plots(f"{picture_path}.png")

In [ ]:
unpadded_sequences = faqs_data["aaSeqCDR3"].tolist()



In [ ]:
Analyse = AnalyseModel(r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model", load_masked_lm=True)
ppl = Analyse.calculate_perplexity(unpadded_sequences)
print(ppl)

### All complementary regions but no framework regions

In [ ]:
import pandas as pd

# Load the data
nanobody_delphia = pd.read_excel(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\nanobody_delphia_tidied.xlsx")
nanobody_delphia = nanobody_delphia.dropna(subset=['CDR1', 'CDR2', 'CDR3', 'Full sequence'])  

def pad_multiple_sequences(fullseq, *subseqs):
    # Create a list of <PAD> for each character in full sequence
    padded_sequence = ['<pad>' for _ in range(len(fullseq))]
    
    # Process each subsequence
    for subseq in subseqs:
        start_index = fullseq.find(subseq)
        if start_index != -1:
            # Replace the <PAD> tokens with the subsequence at the correct position
            for i in range(len(subseq)):
                padded_sequence[start_index + i] = subseq[i]

    # Join the sequence with space
    return ' '.join(padded_sequence)

# Apply the function to each row in DataFrame to include CDR1, CDR2, CDR3
nanobody_delphia['PaddedSequence'] = nanobody_delphia.apply(lambda x: pad_multiple_sequences(x['Full sequence'], x['Framework 1'], x['Framework 2'], x['Framework 3']), axis=1)

# Example of saving or viewing the results
print(nanobody_delphia[['Full sequence', 'PaddedSequence']].head())

In [ ]:
padded_sequences = nanobody_delphia["PaddedSequence"].tolist()
labels = nanobody_delphia["Ag1_modified"].tolist()


In [ ]:
picture_path = "position_all_fr_importance"
title_extension = "specifically trained model"
model_path = r"C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model"
Analyse = AnalyseModel(model_path)
reduced_X = Analyse.embed_cluster_label(padded_sequences, labels, explained_variance_threshold = 0.9, n_neighbors = 15, min_dist = 0.1, alpha = 0.8, cmap = "Set2", title = f"Embedding of position specific padded FR1, FR2 and FR3 sequences.")
reduced_X.to_csv(f"{picture_path}.csv")

Analyse.save_in_plots(f"{picture_path}.png")

### Make barplot with venom composition

In [ ]:
venom_comp = pd.read_excel(r"C:\Users\nilsh\OneDrive\Desktop\results_thesis\data\venom_compmosition.xlsx", index_col = 0)
venom_comp.head(4)

In [ ]:
specific_targets = venom_comp.loc[["Dp4", "Nu6", "Nm8",  "Dp8"]]

In [ ]:
import matplotlib.pyplot as plt

legend_settings = {'loc': 'center left','facecolor': 'black',  'bbox_to_anchor': (1, 0.5), 'ncols': 1, 'fontsize': 16, 'frameon': True, 'framealpha': 1, 'facecolor': 'white', 'mode': None, 'title_fontsize': 'large', 'title_fontsize': 'large'}
# Creating the stacked bar plot
Analyse.make_figure()

ax = specific_targets.plot(kind='bar', stacked=True, figsize=(10, 7), colormap= "tab10", alpha = 0.9, ax = Analyse.ax)
label_settings = {'fontfamily': 'serif',
 'fontsize': '14',
 'fontstyle': 'normal',
 }
# Adding labels and title
plt.xlabel('Venom', **Analyse.font_settings_normal)
plt.ylabel('Composition (%)', **Analyse.font_settings_normal)
plt.title('Toxin Composition by Venom Type', **Analyse.font_settings_title)
plt.xticks(rotation=0)
# Show legend and plot
plt.legend(title='Toxin Types', **legend_settings)
Analyse.update_plot()
Analyse.save_in_plots("venom_composition.png")

In [ ]:
Analyse.font_settings

### Validating with machine learning


In [ ]:
sequencing_report["Experiment"].unique()

In [ ]:


Data = DataPipeline(no_sequences = 1000000)
y = Data.init_sequencing_report['Experiment'].tolist()
y_encoded = [0 for item in y if item == "cLNTX_non-bind"]
y_encoded = [1 for item in y if item == ]
ML = SupervisedML(Data.X, y_encoded, cv_components = 5)
model = ML.logistic_regression()
scores = ML.do_scikits_cv(model)
ML.do_nn_cv()


In [ ]:
Data = DataPipeline(no_sequences = 1000000, model = esm_small)
y = Data.init_sequencing_report['Experiment'].tolist()
y_encoded = [0 if item == "cLNTX_non-bind" else 1 for item in y]
ML = SupervisedML(Data.X, y_encoded, cv_components = 5)
model = ML.logistic_regression()
scores = ML.do_scikits_cv(model)
ML.do_nn_cv()

three class problem: cLNTX ++ and aBGTX-+ | cLNTX +- aBGTX +- | Non bing

In [5]:
chosen_columns = ["cLNTX_non-bind", "cLNTX_++", "aBGTX_-+", "cLNTX_+-", "aBGTX_+-"]
Data = DataPipeline(no_sequences = 1000000, model = nanobody_model, choose_labels= chosen_columns)
y = Data.init_sequencing_report['Experiment'].tolist()


Some weights of EsmModel were not initialized from the model checkpoint at C:\Users\nilsh\my_projects\SeqLP\tests\test_data\nanobody_model and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\nilsh\my_projects\SeqLP\src\seqlp\visualize\load_model.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sequencing_report[['full_sequence', 'CDRPositions']] = sequencing_report.apply(ExtractData.calculate_cdr_positions, axis=1, result_type='expand')
c:\Users\nilsh\my_projects\SeqLP\src\seqlp\visualize\load_model.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Explained variance after reducing to 10 dimensions:0.9186712503433228


ValueError: The number of samples in X and y should be equal

In [16]:
y_encoded = [0 if item == "cLNTX_non-bind" else 1 if item in ['cLNTX_++', 'aBGTX_-+'] else 2 if item in ['cLNTX_+-', 'aBGTX_+-'] else item for item in y]

ML = SupervisedML(Data.X, y_encoded, cv_components = 5)
model = ML.logistic_regression()
scores = ML.do_scikits_cv(model)
ML.do_nn_cv(num_epochs = 10)

Cross-validated scores: [0.65164761 0.65501009 0.6529926  0.67316745 0.64468371]
Average accuracy: 0.6555002937074204
FOLD 0
-------------------------------
Epoch 1
Validation: Avg loss: 0.8985, Accuracy: 0.6543
---------------------------------
Epoch 2
Validation: Avg loss: 0.8334, Accuracy: 0.7034
---------------------------------
Epoch 3
Validation: Avg loss: 0.7938, Accuracy: 0.7189
---------------------------------
Epoch 4
Validation: Avg loss: 0.7671, Accuracy: 0.7323
---------------------------------
Epoch 5
Validation: Avg loss: 0.7475, Accuracy: 0.7424
---------------------------------
Epoch 6
Validation: Avg loss: 0.7327, Accuracy: 0.7471
---------------------------------
Epoch 7
Validation: Avg loss: 0.7214, Accuracy: 0.7492
---------------------------------
Epoch 8
Validation: Avg loss: 0.7126, Accuracy: 0.7492
---------------------------------
Epoch 9
Validation: Avg loss: 0.7055, Accuracy: 0.7485
---------------------------------
Epoch 10
Validation: Avg loss: 0.6996, Acc

In [17]:
chosen_columns = ["cLNTX_non-bind", "cLNTX_++", "aBGTX_-+", "cLNTX_+-", "aBGTX_+-"]
Data = DataPipeline(no_sequences = 1000000, model = esm_small, choose_labels= chosen_columns)
y = Data.init_sequencing_report['Experiment'].tolist()
y_encoded = [0 if item == "cLNTX_non-bind" else 1 if item in ['cLNTX_++', 'aBGTX_-+'] else 2 if item in ['cLNTX_+-', 'aBGTX_+-'] else item for item in y]

ML = SupervisedML(Data.X, y_encoded, cv_components = 5)
model = ML.logistic_regression()
scores = ML.do_scikits_cv(model)
ML.do_nn_cv(num_epochs = 5)

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\nilsh\my_projects\SeqLP\src\seqlp\visualize\load_model.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sequencing_report[['full_sequence', 'CDRPositions']] = sequencing_report.apply(ExtractData.calculate_cdr_positions, axis=1, result_type='expand')
c:\Users\nilsh\my_projects\SeqLP\src\seqlp\visualize\load_model.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index